In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36'}

In [ ]:
response = requests.get('https://latin.packhum.org/browse', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

homepage_links = soup.find_all('a')
homepage_links_references = []
for tag in homepage_links:
    homepage_link_reference = tag.get('href')
    if 'author' in homepage_link_reference:
        homepage_links_references.append(homepage_link_reference)

author_tags = soup.find_all(attrs={"class": "srch"})
author_names = []
for tag in author_tags:
    author_names.append(tag.text)

In [62]:
works = []

for author_index in range(len(homepage_links_references)):
    response = requests.get('https://latin.packhum.org' + homepage_links_references[author_index], headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    works_links = soup.find_all('a')
    works_links_references = []
    for tag in works_links:
        works_links_reference = tag.get('href')
        if 'loc' in works_links_reference:
            works_links_references.append(works_links_reference)

    works_tags = soup.find_all(attrs={"class":"wnam"})
    works_names = []
    for tag in works_tags:
        works_names.append(tag.text)

    for work_index in range(len(works_links_references)):
        works.append([author_names[author_index], works_links_references[work_index], works_names[work_index]])

In [224]:
print(works[0])

['Ablabius  Ablab 2000', '/loc/2000/1/0', 'epigramma']


In [ ]:
texts = []

for work_index in range(len(works)):

    cumulative_text = ""
    page_increment = 0
    multiple_pages = False

    while True:
        response = requests.get('https://latin.packhum.org' + works[work_index][1][:-1] + str(page_increment), headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        full_text = ""
        text_sections = soup.find_all('td')

        for text in text_sections:
            full_text = full_text + " " + text.text.strip()

        if full_text in cumulative_text:
            break

        cumulative_text += " " + full_text

        if page_increment > 1:
            multiple_pages = True

        page_increment += 1

    current_work_copy = works[work_index].copy()
    current_work_copy.append(cumulative_text)
    texts.append(current_work_copy)

In [ ]:
for text in texts:
    text[0] = re.sub(r'\s+', ' ', re.sub('[^a-zA-Z]', ' ', text[0])).strip()
    text[1] = 'https://latin.packhum.org' + text[1]
    word_list = re.sub(r'\s+', ' ', re.sub('[^a-z]', ' ', text[3].lower())).strip().split()
    word_dictionary = {}
    for word in word_list:
        if word in word_dictionary:
            word_dictionary[word] += 1
            continue
        word_dictionary[word] = 1
    text.append(word_dictionary)

AttributeError: 'dict' object has no attribute 'lower'

In [ ]:
df = pd.DataFrame(texts, columns=['author', 'source', 'title', 'text', 'word'])
df['word'] = df['word'].apply(lambda row: list(row.items()))
df = df.explode('word')
df[['word', 'count']] = df['word'].apply(pd.Series)

In [ ]:
display(df)

In [ ]:
"""
test_array = [[' author   ', '/loc/100/1/0', 'title', 'text and     text and text and other  stuff too'], ['an author2 ', '/loc/100/1/1', 'another title', ' words and  text and things and other stuffs 2']]
for text in test_array:
    text[0] = re.sub(r'\s+', ' ', re.sub('[^a-zA-Z]', ' ', text[0])).strip()
    text[1] = 'https://latin.packhum.org' + text[1]
    word_list = re.sub(r'\s+', ' ', re.sub('[^a-z]', ' ', text[3].lower())).strip().split()
    word_dictionary = {}
    for word in word_list:
        if word in word_dictionary:
            word_dictionary[word] += 1
            continue
        word_dictionary[word] = 1
    text.append(word_dictionary)

print(test_array)


df = pd.DataFrame(test_array, columns=['author', 'source', 'title', 'text', 'word'])
df['word'] = df['word'].apply(lambda row: list(row.items()))
df = df.explode('word')
df[['word', 'count']] = df['word'].apply(pd.Series)

display(df)
"""

[['author', 'https://latin.packhum.org/loc/100/1/0', 'title', 'text and     text and text and other  stuff too', {'text': 3, 'and': 3, 'other': 1, 'stuff': 1, 'too': 1}], ['an author', 'https://latin.packhum.org/loc/100/1/1', 'another title', ' words and  text and things and other stuffs 2', {'words': 1, 'and': 3, 'text': 1, 'things': 1, 'other': 1, 'stuffs': 1}]]


,author,source,title,text,word,count
0,author,https://latin.packhum.org/loc/100/1/0,title,text and text and text and other stuff too,text,3
0,author,https://latin.packhum.org/loc/100/1/0,title,text and text and text and other stuff too,and,3
0,author,https://latin.packhum.org/loc/100/1/0,title,text and text and text and other stuff too,other,1
0,author,https://latin.packhum.org/loc/100/1/0,title,text and text and text and other stuff too,stuff,1
0,author,https://latin.packhum.org/loc/100/1/0,title,text and text and text and other stuff too,too,1
1,an author,https://latin.packhum.org/loc/100/1/1,another title,words and text and things and other stuffs 2,words,1
1,an author,https://latin.packhum.org/loc/100/1/1,another title,words and text and things and other stuffs 2,and,3
1,an author,https://latin.packhum.org/loc/100/1/1,another title,words and text and things and other stuffs 2,text,1
1,an author,https://latin.packhum.org/loc/100/1/1,another title,words and text and things and other stuffs 2,things,1
1,an author,https://latin.packhum.org/loc/100/1/1,another title,words and text and things and other stuffs 2,other,1
